In [1]:
%pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from gensim.models.ldamodel import LdaModel

# Load the LDA model
loaded_lda_model = LdaModel.load('/content/drive/MyDrive/Colab Notebooks/alex/models/lda_50_topics/lda_model_50_topics.gensim')

# Now you can access information from the loaded model.
# For example, you can print the topics:
print("Topics:")
for topic_id, topic in loaded_lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# You can also access the dictionary (id2word)
print("\nDictionary:")
print(loaded_lda_model.id2word)

# And the number of topics
print("\nNumber of topics:")
print(loaded_lda_model.num_topics)

Topics:
Topic 2: 0.025*"athletes" + 0.024*"schools" + 0.021*"ncaa" + 0.017*"million" + 0.017*"university" + 0.015*"sports" + 0.015*"school" + 0.014*"money" + 0.014*"college_sports" + 0.013*"football"
Topic 26: 0.029*"olympic" + 0.018*"olympics" + 0.017*"gold" + 0.017*"world" + 0.015*"time" + 0.015*"won" + 0.014*"championships" + 0.012*"medal" + 0.010*"team" + 0.010*"gymnastics"
Topic 11: 0.033*"baseball" + 0.019*"softball" + 0.017*"innings" + 0.016*"college_world_series" + 0.015*"hit" + 0.015*"inning" + 0.015*"regional" + 0.013*"run" + 0.012*"field" + 0.012*"pitcher"
Topic 47: 0.041*"article" + 0.031*"sports" + 0.031*"odds" + 0.023*"news" + 0.022*"originally" + 0.021*"reading" + 0.020*"independently" + 0.019*"influence" + 0.018*"links" + 0.018*"clicking"
Topic 40: 0.212*"johnson" + 0.097*"jackson" + 0.046*"robinson" + 0.038*"van" + 0.033*"riley" + 0.029*"bryant" + 0.024*"walton" + 0.023*"bird" + 0.022*"neal" + 0.022*"cunningham"
Topic 39: 0.054*"coach" + 0.022*"program" + 0.020*"coache

In [3]:
from gensim import corpora

# Load the corpus from the .mm file
corpus = corpora.MmCorpus('/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/corpus.mm')

print("Corpus loaded successfully.")
# You can optionally print the corpus to inspect its structure
# print(list(corpus))

Corpus loaded successfully.


In [4]:
import json
import pandas as pd

# Assuming your JSON file contains a list of lists, where each inner list is a tokenized document
# Replace '/content/drive/MyDrive/path/to/your/texts.json' with the actual path to your JSON file
texts_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json'

with open(texts_file_path, 'r') as f:
    texts = json.load(f)

print(f"Loaded {len(texts)} documents from {texts_file_path}")

Loaded 149649 documents from /content/drive/MyDrive/Colab Notebooks/alex/models/dictionary_and_corpus/texts.json


In [5]:
# import pandas as pd

# # Load the tokenized data from the Excel file
# try:
#     tokenized_data_df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/alex/updated_ tokenized_newspaper_data.xlsx')
#     print("Excel file loaded successfully.")
#     # Assuming the original text is in a column named 'text' or similar.
#     # Please adjust the column name if it's different in your Excel file.
#     if 'Main_text_tokenized' in tokenized_data_df.columns:
#         texts = tokenized_data_df['Main_text_tokenized'].tolist()
#         print(f"Extracted {len(texts)} documents.")
#     else:
#         texts = None
#         print("Could not find a 'text' column in the Excel file. Please make sure the column containing original texts is named 'text' or update the code.")

# except FileNotFoundError:
#     print("Error: The file was not found at the specified path.")
# except Exception as e:
#     print(f"An error occurred while reading the Excel file: {e}")

In [6]:
#TFinding the dominant topic in each sentence

def format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts): # Updated to use loaded_lda_model and corpus, and texts=None
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([sent_topics_df, pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]).to_frame().T], ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

     # Add original text to the end of the output
    if texts is not None: # Added check if texts is not None
        contents = pd.Series(texts)
        sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


# Assuming texts is available as data_words or you provide another list/Series of texts
# df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts) # Updated to use loaded_lda_model and corpus, commented out as data_words is undefined
# To run this cell, define `data_words` with your text data before calling the function.
# For example:
# data_words = ["your document text 1", "your document text 2", ...]
# Then uncomment the line below:
df_topic_sents_keywords = format_topics_sentences(ldamodel=loaded_lda_model, corpus=corpus, texts=texts)

# Format
if 'df_topic_sents_keywords' in locals(): # Added check if df_topic_sents_keywords is defined
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
    # Show
    display(df_dominant_topic.head(20)) # Use display for better formatting

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.2835,"team, players, lot, time, play, playing, coach...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,21,0.2231,"family, school, life, home, father, basketball...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,43,0.2648,"fans, time, people, sports, moment, day, fan, ...","[garden, missed, lonely, nights, season, debut..."
3,3,16,0.1769,"nba, basketball, league, celtics, lakers, bull...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.4094,"league, nfl, season, million, contract, team, ...","[robinson, talk, rangers, stanley, cup, finals..."
5,5,11,0.2789,"baseball, softball, innings, college_world_ser...","[conference, pacific, coach_mike, gillespie, s..."
6,6,5,0.4191,"league, nfl, season, million, contract, team, ...","[rangers, signed, center, michael, nylander, m..."
7,7,5,0.1993,"league, nfl, season, million, contract, team, ...","[gavitt, elected, hall, modest, player, dartmo..."
8,8,10,0.1998,"women, south, basketball, connecticut, final, ...","[mythical, element, senior_guard, seimone, aug..."
9,9,21,0.3457,"family, school, life, home, father, basketball...","[ncaa_women, jim, izard, diana, vines, remembe..."


In [7]:
# Display the first few rows of the dataframe to see the structure
display(df_dominant_topic.head())

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,4,0.2835,"team, players, lot, time, play, playing, coach...","[coaching, living, bonnie, henrickson, phone, ..."
1,1,21,0.2231,"family, school, life, home, father, basketball...","[sidebar, ruston, la, dining, louisiana, cajun..."
2,2,43,0.2648,"fans, time, people, sports, moment, day, fan, ...","[garden, missed, lonely, nights, season, debut..."
3,3,16,0.1769,"nba, basketball, league, celtics, lakers, bull...","[ten, ago, chuck, daly, reclining, alongside, ..."
4,4,5,0.4094,"league, nfl, season, million, contract, team, ...","[robinson, talk, rangers, stanley, cup, finals..."


In [8]:
# Sort documents within each topic by percentage contribution in descending order
sorted_documents_by_topic = df_dominant_topic.sort_values(by=['Dominant_Topic', 'Topic_Perc_Contrib'], ascending=[True, False])

# Display the sorted document indexes for each topic
print("Document Indexes for Each Topic (Sorted by Percentage Contribution):")
for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    print(f"\nTopic {topic_num}:")
    # Get documents for the current topic and select only the 'Document_No' column
    topic_documents = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num]['Document_No']
    # Display the document indexes (first 20 as an example)
    print(topic_documents.tolist()[:20])

Document Indexes for Each Topic (Sorted by Percentage Contribution):

Topic 0:
[40536, 51440, 118640, 69572, 100026, 18715, 85634, 109317, 16581, 91620, 125210, 66098, 32444, 54705, 6403, 61097, 44877, 34010, 47620, 25847]

Topic 1:
[105775, 143357, 98701, 98702, 98705, 102704, 37315, 71814, 63449, 72182, 77342, 74777, 147825, 1488, 41837, 110801, 133983, 143353, 138306, 41808]

Topic 2:
[96487, 86931, 69697, 73748, 89046, 55312, 48272, 97697, 50073, 89542, 72246, 10627, 86921, 21372, 29101, 77075, 37592, 65224, 80907, 37593]

Topic 3:
[77022, 70761, 43636, 44086, 25614, 43348, 83656, 50074, 8994, 53822, 19333, 48384, 17457, 11292, 8995, 83562, 52055, 120101, 17582, 91842]

Topic 4:
[37585, 59253, 34996, 8437, 92550, 131752, 108193, 65980, 97427, 76756, 59849, 47004, 53939, 137868, 108379, 94774, 91712, 141992, 76929, 55253]

Topic 5:
[35493, 120843, 41037, 10113, 126468, 34420, 51765, 11961, 58831, 20047, 29280, 28432, 32491, 75230, 44101, 130920, 106675, 30662, 44099, 44103]

Topic 6

In [9]:
# Get the total number of documents for each dominant topic
topic_counts = df_dominant_topic['Dominant_Topic'].value_counts().sort_index()

print("Total number of documents for each topic:")
display(topic_counts)

Total number of documents for each topic:


,count
Dominant_Topic,
0,4465
1,982
2,5228
3,827
4,9765
5,4434
6,58
7,4543
8,976


In [10]:
# Get the top 100 documents for each topic
top_100_documents_per_topic = pd.DataFrame()

for topic_num in sorted_documents_by_topic['Dominant_Topic'].unique():
    # Get the top 100 documents for the current topic
    top_100 = sorted_documents_by_topic[sorted_documents_by_topic['Dominant_Topic'] == topic_num].head(100)
    # Append to the results DataFrame
    top_100_documents_per_topic = pd.concat([top_100_documents_per_topic, top_100], ignore_index=True)

# Display the top 100 documents for each topic
print("Top 100 Documents for Each Topic (Sorted by Percentage Contribution):")

# Iterate through topics and display the top documents for each
for topic_num in top_100_documents_per_topic['Dominant_Topic'].unique():
    print(f"\n--- Topic {topic_num} ---")
    display(top_100_documents_per_topic[top_100_documents_per_topic['Dominant_Topic'] == topic_num])

Top 100 Documents for Each Topic (Sorted by Percentage Contribution):

--- Topic 0 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,40536,0,0.7691,"school, players, college, recruiting, transfer...","[season, pre, season_final, weeks, ranked, nr,..."
1,51440,0,0.722,"school, players, college, recruiting, transfer...","[school, players, expected, sign, national, le..."
2,118640,0,0.7166,"school, players, college, recruiting, transfer...","[richard, pitino, recruiting, picking, gophers..."
3,69572,0,0.7,"school, players, college, recruiting, transfer...","[recruits, weighing, schools, offers, coach, p..."
4,100026,0,0.6565,"school, players, college, recruiting, transfer...","[tyler, dorsey, remains, committed, signature,..."
...,...,...,...,...,...
95,18652,0,0.5021,"school, players, college, recruiting, transfer...","[steve, carp, review, journal, charlie, spoonh..."
96,70167,0,0.5016,"school, players, college, recruiting, transfer...","[mark, anderson, vegas, review, journal, kathy..."
97,121892,0,0.5009,"school, players, college, recruiting, transfer...","[days, football_team, lost, cal, season_finale..."
98,101022,0,0.5004,"school, players, college, recruiting, transfer...","[shea, patterson, senior, quarterback, img, ac..."



--- Topic 1 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
100,105775,1,0.8923,"p.m, espn, march, saturday, friday, winner, su...","[a.m, a.m, p.m, p.m, friday, june, winner, win..."
101,143357,1,0.886,"p.m, espn, march, saturday, friday, winner, su...","[times, eastern, daylight, knoxville, regional..."
102,98701,1,0.8812,"p.m, espn, march, saturday, friday, winner, su...","[asa, hall_fame, stadium, city, double, elimin..."
103,98702,1,0.8812,"p.m, espn, march, saturday, friday, winner, su...","[asa, hall_fame, stadium, city, double, elimin..."
104,98705,1,0.8812,"p.m, espn, march, saturday, friday, winner, su...","[asa, hall_fame, stadium, city, double, elimin..."
...,...,...,...,...,...
195,64918,1,0.5679,"p.m, espn, march, saturday, friday, winner, su...","[professional, basketball, p.m, indiana, comca..."
196,117279,1,0.5666,"p.m, espn, march, saturday, friday, winner, su...","[ud, arena, dayton, tuesday, march, fairleigh,..."
197,105751,1,0.5661,"p.m, espn, march, saturday, friday, winner, su...","[william, mary, a.m, east, bryant, p.m, doak, ..."
198,5038,1,0.5655,"p.m, espn, march, saturday, friday, winner, su...","[american, p.m, ludwig, field, tickets, record..."



--- Topic 2 ---


...

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
4183,61828,47,0.5482,"article, sports, odds, news, originally, readi...","[wku, ncaa_bid, straight, sbc, win, http, www...."
4184,61202,47,0.503,"article, sports, odds, news, originally, readi...","[read, wku, beat, writer, nick, baumgardner, h..."
4185,65063,47,0.503,"article, sports, odds, news, originally, readi...","[read, wku, beat, writer, nick, baumgardner, h..."
4186,65994,47,0.503,"article, sports, odds, news, originally, readi...","[read, wku, beat, writer, nick, baumgardner, h..."
4187,61420,47,0.4183,"article, sports, odds, news, originally, readi...","[ncaa, aug, sports, page, cornhuskers, selecte..."
4188,65966,47,0.404,"article, sports, odds, news, originally, readi...","[read, wku, beat, writer, nick, baumgardner, h..."
4189,65998,47,0.404,"article, sports, odds, news, originally, readi...","[read, wku, beat, writer, nick, baumgardner, h..."
4190,72468,47,0.404,"article, sports, odds, news, originally, readi...","[read, wku, beat, writer, nick, baumgardner, h..."
4191,73692,47,0.3309,"article, sports, odds, news, originally, readi...","[post, staff, writers, mike, wise, chico, harl..."
4192,21126,47,0.3056,"article, sports, odds, news, originally, readi...","[column, june, sports, section, incorrect, att..."



--- Topic 48 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
4232,20404,48,0.7247,"goals, hockey, goal, net, ice, nhl, play, game...","[boston_college, flying, night, scoring, goals..."
4233,39426,48,0.7159,"goals, hockey, goal, net, ice, nhl, play, game...","[frozen, skating, crowd, bradley, center, nort..."
4234,21652,48,0.7071,"goals, hockey, goal, net, ice, nhl, play, game...","[college_hockey, lukas, dora, scored, winner, ..."
4235,81965,48,0.703,"goals, hockey, goal, net, ice, nhl, play, game...","[ferris, earned, chance, play, ncaa_division, ..."
4236,22613,48,0.6923,"goals, hockey, goal, net, ice, nhl, play, game...","[ecac, semifinals, albany, n.y, center, brenda..."
...,...,...,...,...,...
4327,31982,48,0.5639,"goals, hockey, goal, net, ice, nhl, play, game...","[march_madness, north_dakota, bu, worcester, s..."
4328,306,48,0.5638,"goals, hockey, goal, net, ice, nhl, play, game...","[devils, left, wing, jay, pandolfo, played, bu..."
4329,107214,48,0.5637,"goals, hockey, goal, net, ice, nhl, play, game...","[st, paul, weeks, boston, university, hockey_t..."
4330,44261,48,0.5632,"goals, hockey, goal, net, ice, nhl, play, game...","[hanover, n.h, time, boston, university, faced..."



--- Topic 49 ---


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
4332,57307,49,0.8381,"lacrosse, st, goals, island, navy, stony_brook...","[yearsitesemifinals, attendancefinal, attendan..."
4333,70758,49,0.6507,"lacrosse, st, goals, island, navy, stony_brook...","[penn, boston, university, round, ncaa_tournam..."
4334,57044,49,0.6277,"lacrosse, st, goals, island, navy, stony_brook...","[johns_hopkins, tomorrow, p.m, espn, records, ..."
4335,65721,49,0.6126,"lacrosse, st, goals, island, navy, stony_brook...","[getty, images, photo, red, attack, cornell, m..."
4336,70705,49,0.5897,"lacrosse, st, goals, island, navy, stony_brook...","[local, players, competing, ncaa_lacrosse_tour..."
...,...,...,...,...,...
4427,57062,49,0.3766,"lacrosse, st, goals, island, navy, stony_brook...","[tomorrow, semifinals, foxborough, mass, espn,..."
4428,104935,49,0.3761,"lacrosse, st, goals, island, navy, stony_brook...","[home, team, defeated, saturday, winning, day,..."
4429,81702,49,0.3756,"lacrosse, st, goals, island, navy, stony_brook...","[michele, pataia, scored, goals, lead, host, d..."
4430,94446,49,0.3749,"lacrosse, st, goals, island, navy, stony_brook...","[chelsea, williams, shot, liu, post, hold, hal..."


In [11]:
# Define the path to save the Excel file in your Google Drive
excel_file_path = '/content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_50_topics.xlsx'

# Save the DataFrame to an Excel file
try:
    top_100_documents_per_topic.to_excel(excel_file_path, index=False)
    print(f"Successfully saved the top 100 documents per topic to: {excel_file_path}")
except Exception as e:
    print(f"An error occurred while saving the Excel file: {e}")

Successfully saved the top 100 documents per topic to: /content/drive/MyDrive/Colab Notebooks/alex/top_100_articles/top_100_documents_per_topic_for_50_topics.xlsx


##### **© 2024–2025 MD Rafsun Sheikh**
##### **Licensed under the Apache License, Version 2.0.**